# Rekonstruktion von Baum Strukturen nach Yan et al. (2009)
Dieses Notebook implementiert einen vereinfachten Algorithmus zur Rekonstruktion von Ästen und Skeletten aus 3D-Punktwolken eines Baumes – basierend auf der Methode von Yan et al. (2009).

### 1. Punktwolke (TLS) laden
Hier kannst du eine Punktwolke aus einer `.csv`, `.ply` oder `.las` Datei einlesen. Die Punktwolke muss Spalten mit `x`, `y`, `z` enthalten.

In [9]:
import numpy as np
import pandas as pd

# ======= OPTION A: CSV-Datei =======
# data = pd.read_csv("your_file.csv")
# points = data[['x', 'y', 'z']].to_numpy()

# ======= OPTION B: PLY-Datei =======
# import open3d as o3d
# pcd = o3d.io.read_point_cloud("your_file.ply")
# points = np.asarray(pcd.points)

# ======= OPTION C: LAS-Datei =======
import laspy
las = laspy.read(r"C:\_Data\BTh11\BTh11_Trieblaengenwachstum\07_Check_PG-PointCloud_Completeness\70_BaseDate\PointCloud\20250319_RTC360_Kirsche_clipped.las")
points = np.vstack((las.x, las.y, las.z)).T

print("Punktanzahl:", points.shape)


Punktanzahl: (212935030, 3)


### 2. Segmentierung der Punktwolke

In [10]:
from sklearn.cluster import KMeans

# Passe die Clusteranzahl nach Bedarf an
kmeans = KMeans(n_clusters=5, random_state=0).fit(points)
labels = kmeans.labels_


### 3. Nachbarschaftsgraph erstellen

In [11]:
import networkx as nx
from scipy.spatial import KDTree
from tqdm import tqdm

tree = KDTree(points)
G = nx.Graph()

for i, pt in tqdm(enumerate(points), total=len(points), desc="Baue Nachbarschaftsgraph"):
    idx = tree.query_ball_point(pt, r=0.3)  # Radius ggf. anpassen
    for j in idx:
        if i != j:
            dist = np.linalg.norm(points[i] - points[j])
            G.add_edge(i, j, weight=dist)



Baue Nachbarschaftsgraph:   0%|          | 454/212935030 [25:14<197286:47:44,  3.34s/it]


KeyboardInterrupt: 

### 4. Skelettstruktur (Minimum Spanning Tree)

In [ ]:
mst = nx.minimum_spanning_tree(G)


### 5. Längsten Pfad finden und B-Spline approximieren

In [ ]:
from scipy.interpolate import splprep, splev

# Endpunkte des MST
degrees = dict(mst.degree())
endpoints = [node for node, deg in degrees.items() if deg == 1]

# Längster Pfad im MST
longest_path = []
max_len = 0
for i in endpoints:
    for j in endpoints:
        if i != j and nx.has_path(mst, i, j):
            path = nx.shortest_path(mst, i, j)
            if len(path) > max_len:
                max_len = len(path)
                longest_path = path

# Spline berechnen
path_points = points[longest_path]
tck, u = splprep(path_points.T, s=1.0)
spline = splev(np.linspace(0, 1, 100), tck)
spline_points = np.array(spline).T


### 6. Visualisierung

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points[:,0], points[:,1], points[:,2], c=labels, cmap='tab10', s=5, alpha=0.5)
ax.plot(spline_points[:,0], spline_points[:,1], spline_points[:,2], 'k-', linewidth=2, label='B-Spline Skelett')
ax.set_title("Rekonstruktion aus Punktwolke (.las)")
ax.legend()
plt.show()
